In [1]:
from chebGreen import ChebGreen
import numpy as np

%env TF_MLC_LOGGING=1
%env TF_CPP_MIN_LOG_LEVEL=2

env: TF_MLC_LOGGING=1
env: TF_CPP_MIN_LOG_LEVEL=2


In [2]:
airy = ChebGreen([1,5,10], generateData = True, script = "generate_example", example = "airy_equation")

Generating dataset for example 'airy_equation'
Dataset for for Theta = 1.00. Skipping dataset generation.
Dataset for for Theta = 5.00. Skipping dataset generation.
Dataset for for Theta = 10.00. Skipping dataset generation.
-------------------------------------------------------------------------------

Generating chebfun2 models:
Using tensorflow 2.9.0
Found saved model, Loading model for example 'airy_equation' at Theta = 1.00
Metal device set to: Apple M1 Pro
Learning a chebfun2 model for example 'airy_equation' at Theta = 1.00
Chebfun2 model added for example 'airy_equation' at Theta = 1.00

Found saved model, Loading model for example 'airy_equation' at Theta = 5.00
Learning a chebfun2 model for example 'airy_equation' at Theta = 5.00
Chebfun2 model added for example 'airy_equation' at Theta = 5.00

Found saved model, Loading model for example 'airy_equation' at Theta = 10.00
Learning a chebfun2 model for example 'airy_equation' at Theta = 10.00
Chebfun2 model added for example '

In [3]:
airy.G

{1: chebfun2 object
      domain       rank               corner values
 [0,1] x [0,1]     48       [0.006 0.002 0.001 0.002]
 vertical scale = 0.005689,
 5: chebfun2 object
      domain       rank               corner values
 [0,1] x [0,1]     45       [0.006 0.001 0.001 -0.001]
 vertical scale = 0.005755,
 10: chebfun2 object
      domain       rank               corner values
 [0,1] x [0,1]     46       [0.006 0.001 0.001 -0.007]
 vertical scale = 0.006450}